In [ ]:
%load_ext autoreload
%autoreload 2

#%config IPCompleter.greedy=True

# Rotation curves

First, imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os
import astropy.units as u
from astropy.constants import G

import snapshot_obj 
import subhalo
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(subhalo)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

## Get data

In [ ]:
# Which halo:
gn = 2; sgn = 0

In [ ]:
snap_id = 127
sim_id = "V1_LR_fix"

snap = snapshot_obj.Snapshot(sim_id, snap_id)
idx = snap.index_of_halo(gn, sgn)
    
# Compute rotation curves:
data = {}
pts = ['PartType0', 'PartType1', 'PartType4', 'PartType5', 'All']
for pt, pt_name in zip(['gas', 'dm', 'stars', 'bh', 'all'], pts):
    rot_curves = snap.get_subhalos('Vcirc', 
                                   group='Extended/RotationCurve/{}'\
                                   .format(pt_name))
    sub_offset = snap.get_subhalos('SubOffset', 
                                   group='Extended/RotationCurve/{}'\
                                   .format(pt_name))
    v_circ = rot_curves[:,0] * units.cm.to(units.km)
    radii = rot_curves[:,1] * units.cm.to(units.kpc)
       
    data[pt] = {"Vcirc": v_circ[sub_offset[idx]:sub_offset[idx+1]],
                "Radius": radii[sub_offset[idx]:sub_offset[idx+1]]}
    #print(data[pt])

In [ ]:
# Set some parameters:
x_down = -30; x_up = 600
y_down = 0; y_up = 300

In [ ]:
# Construct saving location:
filename = 'rotation_curve_by_type_gn{}_sgn{}_{}.png'\
                .format(gn, sgn, sim_id)
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots()

# Set axes:
axes.set_xlim(x_down, x_up)
axes.set_ylim(y_down,y_up)

# Set labels:
axes.set_xlabel('$r[\mathrm{kpc}]$', fontsize=16)
axes.set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=16)

# Galaxy text:
axes.set_title('Rotation curve: GN={}, SGN={}'.format(gn,sgn))

for pt, entry in data.items():
    if pt != 'all':
        axes.plot(entry["Radius"], entry["Vcirc"], label=pt)
    else:
        axes.plot(entry["Radius"], entry["Vcirc"], 
                  c='k', linestyle='dashed', label=pt)
    
plt.legend()
plt.tight_layout()

#plt.savefig(os.path.join(path,filename), dpi=200)

In [ ]:
snap_id = 127
sim_id = "V1_LR_curvaton_p084_fix"

snap = snapshot_obj.Snapshot(sim_id, snap_id)
idx = snap.index_of_halo(gn, sgn)
    
# Compute rotation curves:
data = {}
for pt, pt_nums in zip(['gas', 'dm', 'stars', 'bh', 'all'],
                        [[0],[1],[4],[5],[0,1,4,5]]):
    v, r = dataset_compute.compute_rotation_curves(snap, 
                                                   part_type=pt_nums)
    r = r * units.cm.to(units.kpc)
    v = v * units.cm.to(units.km)
        
    data[pt] = {"Vcirc": v[idx], "Radius": r[idx]}